In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from telegram_service import TelegramScraper
import asyncio

# Define the function to run the scraper in Jupyter
async def run_telegram_scraper(channels, time_window_minutes):
    scraper = TelegramScraper()
    await scraper.start()  # Start and authenticate
    result = await scraper.fetch_messages(channels, time_window_minutes)
    await scraper.disconnect()  # Disconnect the client
    return result

# Define the channels and time window
channels = ["From_hebron"]  # Replace with your channels
time_window_minutes = 60  # Last 60 minutes

# Run the scraper and print the result
result = asyncio.run(run_telegram_scraper(channels, time_window_minutes))

# Print the JSON result
import json
print(json.dumps(result, indent=4))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
import json
import asyncio

# Define the function to run the scraper in Jupyter
async def run_telegram_scraper(channels, time_window_minutes):
    scraper = TelegramScraper()
    await scraper.start()  # Start and authenticate
    result = await scraper.fetch_messages(channels, time_window_minutes)
    await scraper.disconnect()  # Disconnect the client
    return result

# Define the channels and time window
channels = ["From_hebron"]  # Replace with your channels
time_window_minutes = 60  # Last 60 minutes

# Run the scraper and get the result
result = asyncio.run(run_telegram_scraper(channels, time_window_minutes))

# Specify the output JSON file path
output_file = "telegram_messages.json"

# Save the result as a local JSON file
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print(f"Messages saved to {output_file}")


RuntimeError: asyncio.run() cannot be called from a running event loop